## Co-folding + Affinity Prediction with Boltz-2

Before running this notebook, please ensure you:

1. Are logged in by running `dm login EMAIL` in the terminal
2. Have a Token registered and saved on the file


In [ ]:
import json
import time
import zipfile

import py3Dmol

import deepmirror.api as api

### Define helper functions

In [ ]:
def cofold(
    chains: list[dict],
    model="boltz2",
    settings: dict[str, str | float | int] | None = None,
) -> str:
    response = api.structure_prediction(
        chains, model=model, constraint_settings=settings
    )
    task_id = response["task_id"]

    while True:
        response = api.get_structure_prediction(task_id)
        status = response["status"]
        if status == "completed":
            break
        print(f"Current status: {status} - Waiting 2 min for completion...")
        time.sleep(120)

    with open(f"result-{task_id}.zip", "wb") as f:
        f.write(api.download_structure_prediction(task_id))

    return task_id


def view_results(task_id: str):
    with zipfile.ZipFile(f"result-{task_id}.zip", "r") as zf:
        cif_data = zf.read("data.cif").decode("utf-8")
        affinity = json.loads(zf.read("affinity_data.json").decode("utf-8"))
    view = py3Dmol.view(width=400, height=400)
    view.addModel(cif_data, "cif")
    view.setStyle({"cartoon": {}})
    view.addStyle({"hetflag": True}, {"stick": {}})
    view.zoomTo()
    return (view, affinity)

### Set the chains you want to co-fold

In [ ]:
chains = [
    {
        "label": "A",
        "value": "MVTPEGNVSLVDESLLVGVTDEDRAVRSAHQFYERLIGLWAPAVMEAAHELGVFAALAEAPADSGELARRLDCDARAMRVLLDALYAYDVIDRIHDTNGFRYLLSAEARECLLPGTLFSLVGKFMHDINVAWPAWRNLAEVVRHGARDTSGAESPNGIAQEDYESLVGGINFWAPPIVTTLSRKLRASGRSGDATASVLDVGCGTGLYSQLLLREFPRWTATGLDVERIATLANAQALRLGVEERFATRAGDFWRGGWGTGYDLVLFANIFHLQTPASAVRLMRHAAACLAPDGLVAVVDQIVDADREPKTPQDRFALLFAASMTNTGGGDAYTFQEYEEWFTAAGLQRIETLDTPMHRILLARRATEPSAVPEGQASENLYFQ",
        "type": "protein",
    },
    {
        "label": "B",
        "value": "N[C@@H](Cc1ccc(O)cc1)C(=O)O",
        "type": "ligand",
    },
]

### Initiate Structure Prediction

In [ ]:
task_id = cofold(chains, model="boltz2", settings={"affinity": "B"})

In [ ]:
view, affinity = view_results(task_id)
print(f"Predicted log IC50: {affinity['affinity_pred_value']:.2f}")
view